<a href="https://colab.research.google.com/github/steflhermitte/EO-Notebooks/blob/master/RemoteSensing%2BBigData/RSBS_Notebook4_GEEinPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" width="200" align='right'>](https://www.tudelft.nl/citg/over-faculteit/afdelingen/geoscience-remote-sensing/staff/scientific-staff/dr-stef-lhermitte)


# GEE in python 
*Created by Stef Lhermitte (s.lhermitte at tudelft.nl / [@steflhermitte](https://twitter.com/StefLhermitte) | www.earthmapps.io)*

*with inspiration and fragments from:*

 *  [geemap](http://www.geemap.org)
 *  [eemont](https://github.com/davemlz/eemont)
 *  [wxee](https://github.com/aazuspan/wxee)

[![Twitter](https://img.shields.io/twitter/url/https/twitter.com/cloudposse.svg?style=social&label=%20%40steflhermitte)](https://twitter.com/steflhermitte)


## Before you start
Before starting this notebook make you need to install the necessary packages. For your local computer, you only need to install the packages once and you won't need the next cell. For Colab, you need to run the following installation steps every time you start a new notebook and restart the environment afterwards. 

In [ ]:
# You need to install them once locally and every time in colab
!pip install geemap
!pip install wxee
!pip install eemont

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

## Load packages + authenticate

In [ ]:
import ee
import geemap
import eemont
import wxee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Geemap add external data

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# Visualize GEE data
landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003') \
    .select([0, 1, 2, 3, 4, 6])
landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.4
}
Map.addLayer(landsat7, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

hyperion = ee.ImageCollection('EO1/HYPERION') \
    .filter(ee.Filter.date('2016-01-01', '2017-03-01'));
hyperion_vis = {
  'min': 1000.0,
  'max': 14000.0,
  'gamma': 2.5,
}
Map.addLayer(hyperion, hyperion_vis, 'EO1/HYPERION');

Explore dataset catalog (gee and curated community collection) and use example code.

In [ ]:
from IPython.display import Video
Video("https://user-images.githubusercontent.com/5016453/184171214-49e58375-196f-4349-9ec1-ae143b63eb42.mp4")

You can also add local vector files. For example, a dataset of all the countries as a shp-file ([Download it here](https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/countries)), which we downloaded and added to the folder `sample_data`

In [ ]:
# Add local vector files
countries_shp = '/content/sample_data/CNTR_BN_60M_2020_4326.shp'
countries = geemap.shp_to_ee(countries_shp)
Map.addLayer(countries, {}, 'Countries')

Similarly you can also load [rasters](https://geemap.org/notebooks/25_load_rasters/), [cloud geotiffs](https://geemap.org/notebooks/38_cloud_geotiff/), etc

# Geemap js to py

In [ ]:
js_snippet = """
// Load an image.
var image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318');

// Define the visualization parameters.
var vizParams = {
  bands: ['B5', 'B4', 'B3'],
  min: 0,
  max: 0.5,
  gamma: [0.95, 1.1, 1]
};

// Center the map and display the image.
Map.setCenter(-122.1899, 37.5010, 10); // San Francisco Bay
Map.addLayer(image, vizParams, 'false color composite');

"""

In [ ]:
geemap.js_snippet_to_pynb(js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True)

# Eemont processing

In [ ]:
point = ee.Geometry.PointFromQuery('Cali, Colombia',user_agent = 'eemont-example') # Extended constructor

S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(point)
    .closest('2020-10-15') # Extended (pre-processing)
    .maskClouds(prob = 70) # Extended (pre-processing)
    .scaleAndOffset() # Extended (pre-processing)
    .spectralIndices(['NDVI','NDWI','BAIS2'])) # Extended (processing)
print(S2.first())

# Geemap export

### Export an image

In [ ]:
image = ee.Image('LE7_TOA_5YEAR/1999_2003')
filename = 'landsat.tif'

In [ ]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon([[[-115.413031, 35.889467],
    [-115.413031, 36.543157],
    [-114.034328, 36.543157],
    [-114.034328, 35.889467],
    [-115.413031, 35.889467]]])
    feature = ee.Feature(geom, {})

roi = feature.geometry()

In [ ]:
image = image.clip(roi).unmask()

geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)

In [ ]:
image = image.clip(roi).unmask()
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=True)

### Export an image collection

In [ ]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2008-01-01', '2020-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))

In [ ]:
out_dir = '/content/sample_data/'

In [ ]:
print(collection.aggregate_array('system:index').getInfo())

In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir)

### Export to numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810') \
  .select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
  [[[-110.8, 44.7],
    [-110.8, 44.6],
    [-110.6, 44.6],
    [-110.6, 44.7]]], None, False)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

In [ ]:
# Scale the data to [0, 255] to show as an RGB image. 
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255*((rgb_img[:, :, 0:3] - 100)/3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()

# Wxee

In [ ]:
a = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('2012-1-1','2012-1-10').wx.to_xarray()
print(a)

In [ ]:
ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('2012-1-1','2012-1-10').wx.to_tif()

In [ ]:
ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('2012-1-1','2012-1-10').wx.to_xarray(path="/content/gridmet.nc")

# Eemont export

In [ ]:
f1 = ee.Feature(ee.Geometry.Point([3.984770,48.767221]).buffer(50),{'ID':'A'})
f2 = ee.Feature(ee.Geometry.Point([4.101367,48.748076]).buffer(50),{'ID':'B'})
fc = ee.FeatureCollection([f1,f2])

S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
   .filterBounds(fc)
   .filterDate('2020-01-01','2021-01-01')
   .maskClouds()
   .scale()
   .index(['EVI','NDVI']))

# By Region
ts = S2.getTimeSeriesByRegion(reducer = [ee.Reducer.mean(),ee.Reducer.median()],
                              geometry = fc,
                              bands = ['EVI','NDVI'],
                              scale = 10)

# By Regions
ts2 = S2.getTimeSeriesByRegions(reducer = [ee.Reducer.mean(),ee.Reducer.median()],
                               collection = fc,
                               bands = ['EVI','NDVI'],
                               scale = 10)

In [ ]:
print(ts)